# Umbrella Sampling


## Introduction 

Umbrella sampling is technique than can be used in molecular dynamics simulations to improve sampling in systems that have high barriers in the free energy landscape. With plain MD simulations the system can remain trapped in free energy minima. With umbrella sampling extra biasing potentials are added to drive the system in the direction of a chosen collective variable (CV). This enables the calculation of free energy profiles in a much more efficient manner than can be done with regular MD alone.

## Theory
This section explains more of the details behind umbrella sampling, refer to [1,2] for full information. The text in this section is from [3].

The free energy of a system in the canonical ensemble is


$F=-k_B T \text{log}(Z),$

where $Z$ is the canonical partition function. For all but the smallest systems $Z$ is computationally intractable to calculate. However, often the most interesting information about a molecular system is given by the differences in the free energy across system sates. A collective variable CV (also called a reaction coordinate) can be defined which is a continuous variable that distinguishes different states. The simplest types of CVs are geometric distances, .e.g. the distance between two molecules, but there are many possibilities. The CV $x$ is a function of the atomic coordinates, $x(r)$, and multiple different realizations of $r$ can map to the same $x$. The probability distribution of the system can be written in terms $x$:


$p(x) \propto \int{ e^{-\beta E(r)} \delta (x-x(r))dr},$


where we have integrated the Boltzmann distribution over all degrees of freedom for each value of $x$. The probability can be turned into a free energy:


$F(x) = -k_B T \text{log} (p(x)) + C,$

where $C$ is a constant and unimportant as we are only interested in $∆F$. In theory, the free energy profile along $x$ could be computed by sampling the system in equilibrium and recording the probability histogram of the values of $x$ which occur. However, for any non-trivial potential energy surface it will take a very long time to achieve sufficient sampling to get an adequately converged histogram — the high energy states will not be sampled.

To enhance the sampling a technique called umbrella sampling can be used. This adds additional biasing potentials $w(x)$ to the system to restrain it at certain values of $x$. The form of $w$ is usually a harmonic term:


$w(x) = \frac{1}{2} k (x-x_0)^2,$


where $k$ is a constant with units of energy per units of $x$ squared. We then run multiple simulations, each restrained with a different $w$, and then combine them to generate a probability distribution that sufficiently samples the whole range of $x$. The process is illustrated figure 1 and explained in more detail as follows.


![umbrella_sampling](umbrella_sampling.svg)

 **Figure 1. Umbrella sampling method to compute a free energy profile.** (a) Multiple biasing potentials are placed across the collective variable $x$. The free energy of the system is the blue curve, in practice this is unknown. (b) Simulations are run for each $w$, these are called windows. The resulting biased probability distributions $P'(x)$ are plotted. (c) The unbiased free energies $F_i$ from each window, they are each offset by a different $C_i$. (d) WHAM is used to combine the windows and compute the free energy curve.



With a biasing potential $w(x(r))$ the potential energy of the system becomes


$E'(r) = E(r) + w((x(r)))$

which leads to a probability distribution (in the canonical ensemble) of


$p'(x) \propto \int e^{(-\beta E + w)} \delta (x - x(r)) dr \propto p(x) e^-\beta w(x),$


and a free energy of


$F'(x) = -k_B T \text{log}(p(x)) + w(x) + C = F(x) + w(x) + C.$

Thus the unbiased free energy $F$ can be obtained by subtracting the biasing potential $w$ from the biased free energy $F'$. However, when more than one biasing window is used the value of $C$ cannot be neglected as it will be different for each window. For multiple biasing windows we have a set of unbiased (but offset by different $C_i$) free energies,


$F_i(x) = -k_B T \text{log}(p'_i) + w_i + C_i.$

To compute $F$ or $P \propto e^{-\beta F}$ over the full range of $x$ the different $F_i(x)$ need to be combined, this can be accomplished by the Weighted Histogram Analysis Method (WHAM) [1]. The WHAM equations are shown in appendix section 2.

## Umbrella sampling simulations

Most umbrella sampling simulations have three main steps:

1. Preparing the windows, this is usually done using Steered MD (SMD, see section 1 in the appendix).  
2. Running the windows, these can be run in parallel.  
3. Analysing the results, typically this involves computing a PMF with WHAM.  

Often you will find in step 3 that the simulations do not produce a nicely converged PMF. You will need to go back to steps 1 and 2 and change some settings, this trial and improvement feedback loop is a normal part of the process. For this tutorial we will use settings already known to work.

## System

This tutorial will use umbrella sampling to compute the free energy profile of the end to end distance of deca-alanine in vacuum. Deca-alanine is commonly used as a toy system [4], its equilibrium structure is a stable alpha-helix. We will start with the alpha-helix structure, as used in [4], perform a steered MD simulation to pull it from helix into a coil, and then run umbrella sampling to compute the free energy profile along the collective variable. Figure 2 shows the initial alpha-helix structure and the final extended coil structure. We will compute the free energy difference between these two states.
![deca-alanine](deca-alanine.png)
**Figure 2** Structure of deca-alanine for different values of the end to end distance $r$.


## Step 1 - setting up the windows with SMD

The script below does the following steps:
-   loads in a PDB file
-   Defines a collective variable between the first and last alpha-carbon
-   Adds a harmonic restraint to the CV
-   Runs a simulation where the location of the harmonic restraint is moved with constant velocity from 1.3nm to 3.3nm (This is called constant velocity steered MD).
-   Saves a configuration for each of the 24 equally spaced windows we have defined.

In [ ]:
import openmm as mm
import openmm.app as app
import openmm.unit as unit
from sys import stdout
import numpy as np

pdb = app.PDBFile("deca-ala.pdb")

forcefield = app.ForceField('amber14-all.xml')

modeller = app.Modeller(pdb.topology, pdb.positions)
#modeller.addHydrogens(forcefield)

# We have a single molecule in vacuum so we use no cutoff.
system = forcefield.createSystem(modeller.topology, nonbondedMethod=app.NoCutoff,constraints=app.HBonds)
integrator = mm.LangevinMiddleIntegrator(300*unit.kelvin, 1/unit.picosecond, 0.002*unit.picoseconds)
simulation = app.Simulation(modeller.topology, system, integrator)
simulation.context.setPositions(modeller.positions)
simulation.reporters.append(app.PDBReporter('smd_traj.pdb', 10000))
simulation.reporters.append(app.StateDataReporter(stdout, 10000, step=True,time=True,
        potentialEnergy=True, temperature=True, speed=True))

# equilibrate
simulation.step(1000)

# define the CV as the distance between the CAs of the two end residues
index1 = 8
index2 = 98
cv = mm.CustomBondForce('r')
cv.addBond(index1, index2)

# now setup SMD

# starting value
r0=1.3 # nm

# force constant
fc_pull=1000.0 # kJ/mol/nm^2

# pulling speed
v_pulling=0.02 # nm / ps

# simulation time step
dt=0.002 # ps

# total number of steps
N=60000

# frequency to increment r0 (1 makes the simulation slow)
M=10

# define a harmonic restraint on the CV
# the location of the restrain will be moved as we run the simulation
# this is constant velocity steered MD
pullingForce = mm.CustomCVForce('0.5 * fc_pull * (cv-r0)^2')
pullingForce.addGlobalParameter('fc_pull', fc_pull)
pullingForce.addGlobalParameter('r0', r0)
pullingForce.addCollectiveVariable("cv",cv)
system.addForce(pullingForce)
simulation.context.reinitialize(preserveState=True)

# pulling loop, update CV target location every 10 steps
# save specific configurations when the current_cv_value first reaches the specified windows
windows = np.linspace(1.3,3.3,24)
window_coords = []
window_index = 0

for i in range(N//M):
    simulation.step(M)
    current_cv_value = pullingForce.getCollectiveVariableValues(simulation.context)
    
    if (i*M)%10000==0:
        print("r0 = ",r0, "r = ",current_cv_value)
    
    # increment the location of the CV based on the pulling velocity
    r0 += v_pulling*dt*M
    simulation.context.setParameter('r0',r0)

    # check if we should save this config as a window starting structure
    if (window_index < len(windows) and current_cv_value >= windows[window_index]):
        window_coords.append(simulation.context.getState(getPositions=True, enforcePeriodicBox=False).getPositions())
        window_index+=1

# save the window structures
i=0
for coords in window_coords:
    outfile = open("window_"+str(i)+".pdb","w")
    app.PDBFile.writeFile(simulation.topology,coords, outfile)
    outfile.close()
    i+=1

Once the script has completed running there will be 24 new pdb files called "window_n.pdb" where n is an integer from 0 to 23. 

We have now have the initial configurations for the umbrella sampling windows.

## Step 2 - running the windows

The script to run the windows is very similar to the script in step 1. The key differences are that we load in an initial structure that corresponds to each specific window, and that the harmonic restraint on the CV does not move. The script below defines a function to run one window.

In [ ]:
import openmm as mm
import openmm.app as app
import openmm.unit as unit
from sys import stdout
import numpy as np

def run_window(N):

    windows = np.linspace(1.3,3.3,24)

    window_index = N

    print("running window", window_index)

    pdb = app.PDBFile("window_"+str(window_index)+".pdb")
    forcefield = app.ForceField('amber14-all.xml')
    system = forcefield.createSystem(pdb.topology, nonbondedMethod=app.NoCutoff,constraints=app.HBonds)
    integrator = mm.LangevinMiddleIntegrator(300*unit.kelvin, 1/unit.picosecond, 0.002*unit.picoseconds)
    simulation = app.Simulation(pdb.topology, system, integrator)
    simulation.context.setPositions(pdb.positions)
    simulation.reporters.append(app.StateDataReporter(stdout, 10000, step=True,time=True,
        potentialEnergy=True, temperature=True, speed=True))

    # define the CV as the distance between the CAs of the two end residues
    index1 = 8
    index2 = 98

    cv = mm.CustomBondForce('r')
    cv.addBond(index1, index2)

    # fixed harmonic restraint at the window location
    r0 = windows[window_index]

    # force constant
    fc_pull=1000.0 # kJ/mol/nm^2

    # total number of steps
    N=100000 # 200 ps

    # record the current CV value every M steps
    M=1000

    # define a harmonic restraint on the CV
    umbrellaForce = mm.CustomCVForce('0.5 * fc_pull * (cv-r0)^2')
    umbrellaForce.addGlobalParameter('fc_pull', fc_pull)
    umbrellaForce.addGlobalParameter('r0', r0)
    umbrellaForce.addCollectiveVariable("cv",cv)
    system.addForce(umbrellaForce)
    simulation.context.reinitialize(preserveState=True)

    
    # run the simulation and record the value of the CV every M steps.
    cv_values=[]
    for i in range(N//M):
        simulation.step(M)

        # get the value of the cv
        current_cv_value = umbrellaForce.getCollectiveVariableValues(simulation.context)
        cv_values.append([i, current_cv_value[0]])

    # save the CV timeseries to a file so we can postprocess 
    np.savetxt("cv_values_window_"+str(window_index)+".txt",np.array(cv_values))

    print("Completed window", window_index)

We then run all 24 windows

In [ ]:
for n in range(24):
    run_window(n)

Once all the window simulations have completed you will have the CV timeseries files: "cv_values_window_n.txt"

## Step 3 - analysis - compute the PMF

The first thing to check is that the histograms of the CV timeseries have good overlap. Here is an example:
![histogram](hist.png)


You can plot yours with the script below
(This script also produces the metadata file we will need for the next step).

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# plot the histograms
metafilelines=[]
windows = np.linspace(1.3,3,24)
for i in range(len(windows)):
    data=np.loadtxt("cv_values_window_"+str(i)+".txt")
    plt.hist(data[:,1])
    metafileline= "cv_values_window_"+str(i)+".txt "+str(windows[i])+ " 1000\n"
    metafilelines.append(metafileline)

plt.xlabel("r (nm)")
plt.ylabel("count")

with open("metafile.txt", "w") as f:
    f.writelines(metafilelines)

To compute the PMF we can use WHAM [1]. An easy to use and widely compatible implementation is the WHAM program by Alan Grossfield, it can be downloaded here: http://membrane.urmc.rochester.edu/?page_id=126. It is a C program so it will need to be compiled, the command below should work on Linux and Mac.

In [ ]:
!wget http://membrane.urmc.rochester.edu/sites/default/files/wham/wham-release-2.0.11.tgz
!tar xf wham-release-2.0.11.tgz
!cd wham/wham && make

To use `wham` we need a metadata file that lists the names of each CV timeseries file, the location of the harmonic restraints, and the value of the spring constant. We created this in the histogram plotting script earlier.

The `wham` program is run using command line arguments, read the documentation to find out more: http://membrane.urmc.rochester.edu/sites/default/files/wham/doc.pdf

The command below will compute the PMF from our data. The command line arguments corresponds to a range of 1.3nm to 3.3nm, 50 bins, a tolerance of 1e-6, and a temperature of 300K.

In [ ]:
!./wham/wham/wham 1.3 3.3 50 1e-6 300 0 metafile.txt pmf.txt > wham_log.txt

We can then plot the computed PMF. It should look something like this:
![pmf.png](pmf.png)

In [ ]:
# plot the pmf
pmf=np.loadtxt("pmf.txt")
plt.plot(pmf[:,0], pmf[:,1])
plt.xlabel("r (nm)")
plt.ylabel("PMF (kJ/mol)")
plt.show()

## References

[1] Kumar, S., Rosenberg, J.M., Bouzida, D., Swendsen, R.H. and Kollman, P.A. (1992), *The weighted histogram analysis method for free-energy calculations on biomolecules. I. The method.* J. Comput. Chem., 13: 1011-1021. https://doi.org/10.1002/jcc.540130812  
[2] Kästner, J. (2011), *Umbrella sampling.* WIREs Comput Mol Sci, 1: 932-942. https://doi.org/10.1002/wcms.66  
[3] Farr, S. (2021), https://doi.org/10.17863/CAM.72078  
[4] Park, S., Khalili-Araghi, F., Tajkhorshid, E., and Schulten, K. (2003), *Free energy calculation from steered molecular dynamics simulations using Jarzynski’s equality.* J. Chem. Phys., 119 (6): 3559–3566. https://doi.org/10.1063/1.1590311  

## Appendix

### Steered MD

Steered MD is an enhanced simulation method in its own right. It is a non-equilibrium method which means great care must be taken to ensure the results you get are meaningful. In this tutorial steered MD is just used in the setup phase, we do not calculate any properties with it.
In constant velocity steered MD the biasing potential has the form


$w = \frac{1}{2} k (x-x_0(t))^2, 
x_0(t) = x_0(0)+vt,$

which is just a moving version of the umbrella window bias potential.


### WHAM equations
The WHAM equations [1] are


$P(x_j) = \frac{\sum^{N_w}_i h_i(x_j)}{\sum^{N_w}_i n_i e^ {\beta(C_i - w_i(x_j))} },$


$C_i = -k_B T \log\left( \sum_j P(x_j) e^{-\beta w_i(x_j)} \right),$

where they have been written in a fully discretized form. $P(x_j)$ is the resulting unbiased probability distribution, $j$ is the index for the discrete set of $x_j$ that $P$ is computed over. $N_w$ is the number of windows, $i$ is the index of each window, $n_i$ is the number of data points (realizations of $x$) in the $i$-th window trajectory, $h_i(x_j)$ is the number of points in histogram bin $j$ from trajectory $i$, and  $w_i$ are the biasing potentials. Both equations depend on each other so must be solved self-consistently. In practice this is solved iteratively: initial guesses of $C_i$ are chosen, $P$ is then calculated using the first equation, new values of $C_i$ are then calculated using the second equation, and the process is iterated until the differences between successive values are sufficiently small.
